<a href="https://colab.research.google.com/github/NishaMurarka11/legal_NER/blob/main/Alterate_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/legal_NER

/content/drive/MyDrive/legal_NER


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding, Bidirectional
import pandas as pd
import numpy as np
from keras.layers import LSTM
from keras.layers import TimeDistributed, Activation, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support

In [ ]:
import json
 
train_judgement_data =  open('dataset/train_data/NER_TRAIN_JUDGEMENT.json', 'r') 
judgement_data = json.load(train_judgement_data)
print(len(judgement_data))

train_preamble_data = open('dataset/train_data/NER_TRAIN_PREAMBLE.json', 'r') 
preamble_data = json.load(train_preamble_data)
print(len(preamble_data))

9435
1560


Data Visualization

In [ ]:
judgement_annotations = pd.DataFrame.from_dict(pd.json_normalize(judgement_data[0]['annotations'][0]['result']), orient='columns')
judgement_annotations

,id,from_name,to_name,type,value.start,value.end,value.text,value.labels
0,C8HPTIM1,label,text,labels,90,103,Hongkong Bank,[ORG]
1,KOWE3RAM,label,text,labels,267,278,Rahul & Co.,[ORG]


In [ ]:
preamble_annotations = pd.DataFrame.from_dict(pd.json_normalize(preamble_data[0]['annotations'][0]['result']), orient='columns')
preamble_annotations

,id,from_name,to_name,type,value.start,value.end,value.text,value.labels
0,2JN4GRYC,label,text,labels,7,40,High Court Of Kerala At Ernakulam,[COURT]
1,SIMZ3AEN,label,text,labels,73,83,T.R.Ajayan,[PETITIONER]
2,S9JF9CCI,label,text,labels,171,182,M.Ravindran,[RESPONDENT]
3,XUXELOF9,label,text,labels,237,251,Nirmala Dinesh,[RESPONDENT]
4,60NAL9QX,label,text,labels,304,311,A.Kumar,[LAWYER]
5,J5CXTLAN,label,text,labels,350,365,M.K.Pushpalatha,[LAWYER]
6,0II679BV,label,text,labels,391,400,P.R.Raman,[JUDGE]
7,G17U4A5Z,label,text,labels,425,436,V.K.Mohanan,[JUDGE]


Splitting the data

In [ ]:
!python -m spacy download en_core_web_lg


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 777.4 MB 5.5 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!python -m spacy train training/cfg_albert.cfg --output ./output --paths.train training/Combined_Data_Judgement_Preamble_spacy_format/train.spacy --paths.dev training/Combined_Data_Judgement_Preamble_spacy_format/dev.spacy --gpu-id 0

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-12-11 22:22:01,236] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-11 22:22:01,250] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-12-11 22:22:01,255] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-11 22:22:01,258] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Downloading: 100% 684/684 [00:00<00:00, 537kB/s]
Downloading: 100% 742k/742k [00:00<00:00, 5.60MB/s]
Downloading: 100% 1.25M/1.25M [00:00<00:00, 5.97MB/s]
Downloading: 100% 45.2M/45.2M [00:01<00:00, 38.2MB/s]
Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.

In [ ]:
import spacy
import urllib
albert_ner = spacy.load("output/model-best")
classes= ['COURT', "PETITIONER", "RESPONDENT", "JUDGE", "LAWYER", "DATE", "ORG", "GPE", 
                              "STATUTE", "PROVISION", "PRECEDENT", "CASE_NUMBER", "WITNESS", "OTHER_PERSON"]
judgment_text = urllib.request.urlopen('https://raw.githubusercontent.com/OpenNyAI/Opennyai/master/samples/sample_judgment1.txt').read().decode()
doc = albert_ner(judgment_text)
colors = {'COURT': "#bbabf2", 'PETITIONER': "#f570ea", "RESPONDENT": "#cdee81", 'JUDGE': "#fdd8a5",
              "LAWYER": "#f9d380", 'WITNESS': "violet", "STATUTE": "#faea99", "PROVISION": "yellow",
              'CASE_NUMBER': "#fbb1cf", "PRECEDENT": "#fad6d6", 'DATE': "#b1ecf7", 'OTHER_PERSON': "#b0f6a2",
              'ORG': '#a57db5', 'GPE': '#7fdbd4'}
options = {"ents": classes, "colors": colors}
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
!python -m spacy evaluate model-best training/Combined_Data_Judgement_Preamble_spacy_format/dev.spacy --output outut/metrics.json --gold-preproc

100%|██████████| 7052/7052 [00:06<00:00, 1120.85it/s]


In [5]:
!python -m spacy train training/tok2vec.cfg --output ./output_tok --paths.train training/Combined_Data_Judgement_Preamble_spacy_format/train.spacy --paths.dev training/Combined_Data_Judgement_Preamble_spacy_format/dev.spacy 

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-12 02:02:21.337647: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Created output directory: output_tok
ℹ Saving to output directory: output_tok
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-12-12 02:02:22,554] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-12 02:02:22,567] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-12-12 02:02:22,571] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-12 02:02:22,573] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-12-12 02:02:49,814] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
INFO:spacy:Initialized pipeline component

In [6]:
import spacy
import urllib
tok2_vec_ner = spacy.load("output_tok/model-best")
classes= ['COURT', "PETITIONER", "RESPONDENT", "JUDGE", "LAWYER", "DATE", "ORG", "GPE", 
                              "STATUTE", "PROVISION", "PRECEDENT", "CASE_NUMBER", "WITNESS", "OTHER_PERSON"]
judgment_text = urllib.request.urlopen('https://raw.githubusercontent.com/OpenNyAI/Opennyai/master/samples/sample_judgment1.txt').read().decode()
colors = {'COURT': "#bbabf2", 'PETITIONER': "#f570ea", "RESPONDENT": "#cdee81", 'JUDGE': "#fdd8a5",
              "LAWYER": "#f9d380", 'WITNESS': "violet", "STATUTE": "#faea99", "PROVISION": "yellow",
              'CASE_NUMBER': "#fbb1cf", "PRECEDENT": "#fad6d6", 'DATE': "#b1ecf7", 'OTHER_PERSON': "#b0f6a2",
              'ORG': '#a57db5', 'GPE': '#7fdbd4'}
doc = tok2_vec_ner(judgment_text)
options = {"ents": classes, "colors": colors}
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [10]:
!python -m spacy evaluate output_tok/model-best training/Combined_Data_Judgement_Preamble_spacy_format/dev.spacy --output output_tok/metrics.json --gold-preproc

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-12 02:31:42.855198: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   74.30 
NER R   71.51 
NER F   72.88 
SPEED   21149 


=============================== NER (per type) ===============================

                   P       R       F
COURT          88.03   81.72   84.76
PETITIONER     61.20   65.88   63.46
LAWYER         89.13   84.27   86.63
RESPONDENT     69.28   40.77   51.33
JUDGE          86.61   71.43   78.29
STATUTE        82.79   83.18   82.98
WITNESS        74.00   66.07   69.81
OTHER_PERSON   64.41   69.08   66.67
PRECEDENT      51.80   64.86   57.60
ORG            53.12   44.74   48.57
GPE            45.23   50.56   47.75
DA